# Combinações de bandas para visualização de corpos hídricos com Landsat-8

A maneira mais simples e rápida de identificarmos corpos hídricos nas imagens de satélite é fazer o uso de combinação de bandas espectrais. Para isto utilizaremos os dados do [Landsat-8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2) e aplicaremos as seguintes combinações:


---
`1. Natural Color (Água Transparente)`:
*   [Red, Green, Blue] → [B4, B3, B2]
*   Utilizado para destacar  **Vegetação e Água**
*   **Água limpa**: Azul claro/transparente
*   **Água turva**: Marrom/Verde
---
`2. Falsa Cor para Água (RGB)`:
*   [SWIR1, NIR, Green] → [B6, B5, B3]:
*   Utilizado para destacar **Água Limpa/Lagos**
*   **Água aparece**: Preta/Azul escuro
*   **Vegetação**: Verde claro
*   **Solo**: Amarelo
---
`3. Falsa Cor para Inundações`:
*   [SWIR2, SWIR1, Red] → [B7, B6, B4]
*   Utilizado para destacar **Rios/Água Turva**
*   **Água**: Azul escuro/Preto
*   **Vegetação inundada**: Vermelho escuro
*   **Áreas urbanas**: Amarelo claro
---
`4. Falsa Cor Infravermelha`:
*   [NIR, Green, Blue] → [B5, B4, B3]
*   Utilizado para  destacar **Vegetação e Áreas Úmidas**
*   **Água**: Azul/Preto
*   **Vegetação**: Vermelho
---

# Inicializando o GEE

In [ ]:
# importando o GEE e Geemap
import ee
import geemap

# inicializando GEE
geemap.ee_initialize(project='ee-enrique')

# Carregando os dados e aplicando o fator de escala

In [ ]:
# selecionando o contorno do Estado do Rio Grande do Sul
rs = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM1_NAME', 'Rio Grande Do Sul'))

# aplica fator de escala
def fator_escala(img):

    # Aplica fatores de escala
    opticas = img.select('SR_B.*').multiply(2.75e-05).add(-0.2)
    termais = img.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Retorna imagem com bandas escaladas
    return (img.addBands(opticas, None, True)
               .addBands(termais, None, True)
               .copyProperties(img, ['system:time_start'])
               .set('date', img.date().format('YYYY-MM-dd')))

# carrega os dados
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
             .filterBounds(rs) \
             .filterDate('2024-01-01','2025-01-01') \
             .filter(ee.Filter.lt('CLOUD_COVER', 5)) \
             .map(fator_escala)

# mostra os dados
landsat8

# Plotando mapa

In [ ]:
# parâmetros de visualização
param_vis_1 = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.03, 'max': 0.19}
param_vis_2 = {'bands': ['SR_B6', 'SR_B5', 'SR_B3'], 'min': 0.03, 'max': 0.19}
param_vis_3 = {'bands': ['SR_B7', 'SR_B6', 'SR_B4'], 'min': 0.03, 'max': 0.19}
param_vis_4 = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': 0.03, 'max': 0.19}

# monta painel com 4 mapas
geemap.linked_maps(rows = 2,
                   cols = 2,
                   height = "400px",
                   center = [-29.94, -51.32],
                   zoom = 11,
                   ee_objects = [landsat8.mean().clip(rs), landsat8.mean().clip(rs), landsat8.mean().clip(rs), landsat8.mean().clip(rs)],
                   vis_params = [param_vis_1, param_vis_2, param_vis_3, param_vis_4],
                   labels = ['Natural Color (Água Transparente)', 'Falsa Cor para Água (RGB)', 'Falsa Cor para Inundações', 'Falsa Cor Infravermelha'],
                   label_position = "topright")